In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pprint

In [23]:
dfs = []
for infile in glob.glob('./raw/*.csv'):
    df = pd.read_csv(infile, names=['date', 'a', 'b', 'high', 'close', 'low', 'open', 'volume', '1'])
    df.index=pd.to_datetime(df.date)
    df = df[['high', 'low', 'open', 'close', 'volume']]
#     df = df['close']
    name = os.path.splitext((os.path.basename(infile)))[0]
    if not name.endswith('USD'):
        df[['high', 'low', 'open', 'close']] = 1 / df[['high', 'low', 'open', 'close']]
        df.name = name[-3:] + '_' + 'USD'
        df[['high']], df[['low']] = df[['low']], df[['high']]
#         df[]
    else:
        df.name = name
    dfs.append(df)
    print(df.shape)

(130575, 5)
(130494, 5)
(130317, 5)
(130696, 5)


In [24]:
l = []
for df in dfs:
    l.append(len(df))

mi = dfs[np.argmax(l)].index.copy()
for i in range(len(dfs)):
    name = dfs[i].name
    dfs[i] = dfs[i].reindex(mi)
    dfs[i] = dfs[i].interpolate()
    dfs[i] = dfs[i].fillna(method='bfill', limit=1)
    print(np.where(np.isnan(dfs[i]))[0])
    dfs[i].name = name
    print(dfs[i].name)

[]
JPY_USD
[]
GBP_USD
[]
CAD_USD
[]
EUR_USD


In [26]:
df = pd.concat(dfs, axis=1, keys=[df.name for df in dfs], names=['Currency', 'Price'])
# df = pd.concat(dfs, axis=1)
print(df[:10])

Currency              JPY_USD                                       GBP_USD  \
Price                    high       low      open     close volume     high   
date                                                                          
2007-07-01 19:00:00  0.008127  0.008119  0.008119  0.008121   11.0  2.00870   
2007-07-01 19:30:00  0.008121  0.008121  0.008121  0.008121   28.0  2.00765   
2007-07-01 20:00:00  0.008122  0.008118  0.008121  0.008118   66.0  2.00765   
2007-07-01 20:30:00  0.008123  0.008118  0.008118  0.008122   60.0  2.00905   
2007-07-01 21:00:00  0.008134  0.008121  0.008122  0.008131  184.0  2.00900   
2007-07-01 21:30:00  0.008133  0.008125  0.008131  0.008127  131.0  2.00910   
2007-07-01 22:00:00  0.008127  0.008124  0.008127  0.008125  112.0  2.00890   
2007-07-01 22:30:00  0.008126  0.008121  0.008125  0.008121  132.0  2.00885   
2007-07-01 23:00:00  0.008128  0.008118  0.008121  0.008127  250.0  2.00860   
2007-07-01 23:30:00  0.008130  0.008114  0.008127  0

In [27]:
assert np.isfinite(df.values).all()

In [28]:
# line = [0.18, 0.2, 0.38, 0.4, 0.58, 0.6, 0.78, 0.8, 0.98]

# c = []
# data = []
# # print(df.index)
# for l in line:
#     c.append(df.index[int(len(df.index)*l)])

c1 = df.index[int(len(df.index)*0.6)]
c2 = df.index[int(len(df.index)*0.7)]

df_train_1 = df[df.index<c1]
df_test_1 = df[c1:c2]

# df_train_1 = df[df.index<c[0]]
# df_test_1 = df[c[0]:c[1]]
# df_train_1.to_hdf('./forex_30m_CLOSE_O_1.hf', key='train', mode='w', append=False)
# df_test_1.to_hdf('./forex_30m_CLOSE_O_1.hf', key='test', mode='a', append=False)


# c = df.index[int(len(df.index)*0.9)]
# df_train = df[df.index<c]
# df_test = df[df.index>=c]
# print(df_train.head)
df_train_1.to_hdf('./forex_5f_1.hf', key='train', mode='w', append=False)
df_test_1.to_hdf('./forex_5f_1.hf', key='test', mode='a', append=False)
print(df_train_1)
print(df_test_1)

Currency              JPY_USD                                        GBP_USD  \
Price                    high       low      open     close  volume     high   
date                                                                           
2007-07-01 19:00:00  0.008127  0.008119  0.008119  0.008121    11.0  2.00870   
2007-07-01 19:30:00  0.008121  0.008121  0.008121  0.008121    28.0  2.00765   
2007-07-01 20:00:00  0.008122  0.008118  0.008121  0.008118    66.0  2.00765   
2007-07-01 20:30:00  0.008123  0.008118  0.008118  0.008122    60.0  2.00905   
2007-07-01 21:00:00  0.008134  0.008121  0.008122  0.008131   184.0  2.00900   
2007-07-01 21:30:00  0.008133  0.008125  0.008131  0.008127   131.0  2.00910   
2007-07-01 22:00:00  0.008127  0.008124  0.008127  0.008125   112.0  2.00890   
2007-07-01 22:30:00  0.008126  0.008121  0.008125  0.008121   132.0  2.00885   
2007-07-01 23:00:00  0.008128  0.008118  0.008121  0.008127   250.0  2.00860   
2007-07-01 23:30:00  0.008130  0.008114 

In [30]:
c3 = df.index[int(len(df.index)*0.8)]
c4 = df.index[int(len(df.index)*0.95)]
c5 = df.index[int(len(df.index)*0.99)]

df_train_2 = df[c3:c4]
df_test_2 = df[c4:c5]

df_train_2.to_hdf('./forex_5f_2.hf', key='train', mode='w', append=False)
df_test_2.to_hdf('./forex_5f_2.hf', key='test', mode='a', append=False)
print(df_train_2)
print(df_test_2)

Currency              JPY_USD                                        GBP_USD  \
Price                    high       low      open     close  volume     high   
date                                                                           
2015-08-31 05:30:00  0.008259  0.008253  0.008257  0.008255   707.0  1.54328   
2015-08-31 06:00:00  0.008261  0.008251  0.008255  0.008257   568.0  1.54372   
2015-08-31 06:30:00  0.008261  0.008253  0.008257  0.008253   442.0  1.54324   
2015-08-31 07:00:00  0.008256  0.008248  0.008253  0.008254   493.0  1.54266   
2015-08-31 07:30:00  0.008255  0.008250  0.008254  0.008252   414.0  1.54151   
2015-08-31 08:00:00  0.008258  0.008249  0.008252  0.008249   532.0  1.54226   
2015-08-31 08:30:00  0.008252  0.008243  0.008249  0.008246   687.0  1.54300   
2015-08-31 09:00:00  0.008250  0.008240  0.008246  0.008244   552.0  1.54266   
2015-08-31 09:30:00  0.008249  0.008242  0.008244  0.008243   374.0  1.54105   
2015-08-31 10:00:00  0.008249  0.008243 

In [8]:
c6 = df.index[int(len(df.index)*0.5)]
c7 = df.index[int(len(df.index)*0.8)]
c8 = df.index[int(len(df.index)-1)]

df_train_3 = df[c6:c7]
df_test_3 = df[c7:c8]

df_train_3.to_hdf('./forex_3f_3.hf', key='train', mode='w', append=False)
df_test_3.to_hdf('./forex_3f_3.hf', key='test', mode='a', append=False)
print(df_train_3)
print(df_test_3)

Currency              JPY_USD                      GBP_USD                    \
Price                    high       low     close     high      low    close   
date                                                                           
2012-07-06 00:00:00  0.012521  0.012509  0.012512  1.55350  1.55254  1.55272   
2012-07-06 00:30:00  0.012512  0.012503  0.012503  1.55276  1.55200  1.55200   
2012-07-06 01:00:00  0.012508  0.012497  0.012502  1.55262  1.55188  1.55222   
2012-07-06 01:30:00  0.012506  0.012500  0.012504  1.55266  1.55174  1.55174   
2012-07-06 02:00:00  0.012512  0.012504  0.012511  1.55238  1.55174  1.55200   
2012-07-06 02:30:00  0.012513  0.012509  0.012511  1.55272  1.55184  1.55258   
2012-07-06 03:00:00  0.012512  0.012507  0.012510  1.55284  1.55230  1.55266   
2012-07-06 03:30:00  0.012510  0.012506  0.012508  1.55284  1.55242  1.55270   
2012-07-06 04:00:00  0.012514  0.012506  0.012510  1.55312  1.55270  1.55290   
2012-07-06 04:30:00  0.012514  0.012509 